In [ ]:
# import sqlite3
import pandas as pd
import torch
from tqdm import tqdm
import accelerate

In [ ]:
import os
os.environ["HUGGINGFACE_TOKEN"] = "YOUR TOKEN"

from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
max_memory_map = {
    0: "14GiB", # Или "14000MiB"
    1: "14GiB"  # Или "14000MiB"
    # Если бы была еще CPU память или диск, можно добавить:
    # "cpu": "30GiB"
}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto",
    max_memory=max_memory_map,
)


In [ ]:
system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупаем актив/бычий), 0 (нейтральный), -1 (продаем актив /медвежий)

Проанализируй пост и верни ТОЛЬКО число"""

query = 'Полетела 💩💩💩💩'

In [ ]:
import re

full_prompt = f"""System: {system_prompt}
User: {query}
Assistant:""" # Добавляем маркер начала ответа ассистента

print("Сформированный промпт:")
print(full_prompt)
print("-" * 20)


Сформированный промпт:
System: Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупаем актив/бычий), 0 (нейтральный), -1 (продаем актив /медвежий)

Проанализируй пост и верни ТОЛЬКО число
User: Полетела 💩💩💩💩
Assistant:
--------------------


In [ ]:
messages = [
    # {"role": "system", "content": system_prompt},
    {"role": "user", "content": full_prompt}
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, return_tensors="pt"
).to("cuda")

outputs = model.generate(input_ids, max_new_tokens=1024)
print(tokenizer.decode(outputs[0][input_ids.size(1) :], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-1


In [ ]:
posts_ygpt_9500 = pd.read_excel("/kaggle/input/y-gpt13500/y_gpt_9000.xlsx")
posts_ygpt_9500

,post_id,processed_posts
0,d1a4fb9d-92c9-4f09-83e4-49bb11282ba5,{$LKOH} секта 4800 рассеялась😆
1,f10ce454-8e8b-4ce4-998b-4d582f3aafa8,{$MTLR} о опять мотороллер пампят 😃😃😃
2,1ae3b93d-51eb-4d0c-8cc0-cdaa3b51c541,{$LKOH} когда ливы должны будут прийти примерно?
3,5f500181-6d6a-48b0-b4c4-857a6fa59ed2,"{$MTLR} Ох, знатно ноябрь начнётся на следующе..."
4,0958294a-f507-45b1-8149-bb5e9dcf961d,{$LKOH} ну пугайте так
...,...,...
13495,ed67e8fa-4928-4cd4-b5e2-39d09ea816f1,"{$LKOH} когда нибудь хомяки поймут,что компани..."
13496,6eb52f1a-559c-4567-a1d4-845313d2bb97,"{$SBER} чуть вниз и паника, смешно 🤣"
13497,7a238f49-12cb-4061-9fa7-12578f6ac285,"Аналитики {$SBER} кстати считают, что ставку п..."
13498,97a9e14b-50e3-461a-aed6-c79b211036fd,{$SGZH} \nПолетела 💩💩💩💩


In [ ]:
# posts_ygpt_6500 = pd.read_csv("/kaggle/input/ygpt-data/ygpt_data_raw.csv")
# posts_ygpt_6500

,post_id,processed_posts
0,e7d0b9ff-c742-47dd-951d-910e3b810259,"{$SBER} ..................⬆️300,0 вопрос тольк..."
1,cb5e98e8-820c-4bca-be59-9aef2c4106ad,{$SBER} на чем летим?
2,80176185-ce8b-4875-8c36-51b944cc6833,"{$SBER} как прекрасен шортокрыл, посмотри....\..."
3,521fa124-b357-438b-98e5-e3f0580bdd24,✅ 10 января Лукойл {$LKOH} рассмотрит итоги 20...
4,ea8c0f0a-6b2d-4c5b-bd9b-a1dc41c349ad,"{$SBER}\n \nЧто быстрее, скорость света, или с..."
...,...,...
6495,e24a3eac-31d8-4471-8ea9-f90f11e786fa,{$SGZH} \nА ведь кто то это скупает..\nИли про...
6496,4f9a341d-2a85-4f2a-b357-bfe6fa0567c6,"Мечел -0,9% 454 | 1\nНоватэк -2,9% ..."
6497,1d466170-60bf-4f82-8284-e92ec8ddbec3,{$LKOH} сделают всё чтобы шортики не вылезли д...
6498,2aa8f0a9-621a-41df-b1f9-2a2247aed08b,{$SGZH} хватит губить деревья ради денег! Это ...


In [ ]:
# system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
# Твоя задача - классифицировать пост по его тональности относительно торговли акциями.

# Категории и правила:
# *   **1 (Покупка/Бычий):** Пост явно выражает намерение купить, решение о покупке, удержание лонга с ожиданием роста, или описывает сильные позитивные факторы, прямо указывающие на вероятный рост цены акции. Ключевые слова/идеи: "купил", "докупаю", "лонг", "ракета", "рост", "пробой вверх", "отчет супер", "пора брать", "держать дальше", "потенциал есть".
# *   **-1 (Продажа/Медвежий):** Пост явно выражает намерение продать, решение о продаже, открытие/удержание шорта с ожиданием падения, или описывает сильные негативные факторы, прямо указывающие на вероятное падение цены акции. Ключевые слова/идеи: "продал", "шорт", "сливаю", "падение", "дно не найдено", "отчет плохой", "пора выходить", "фиксация убытка", "коррекция".
# *   **0 (Нейтральный):** Пост НЕ содержит явного торгового сигнала на покупку или продажу. Сюда относятся:
#     *   Вопросы о цене или прогнозах ("что думаете?", "куда пойдет?").
#     *   Констатация фактов или новостей без явной оценки влияния на цену ("вышел отчет", "сегодня дивгэп").
#     *   Смешанные сигналы или сомнения ("вроде растет, но страшно", "с одной стороны..., с другой...").
#     *   Общие рыночные рассуждения без привязки к конкретному действию.
#     *   Фиксация прибыли или убытка *без явного прогноза* дальнейшего движения ("закрыл позицию", "вышел в ноль").
#     *   Неясные или неинформативные сообщения.


# Верни ТОЛЬКО ОДНО число: 1, 0 или -1."""

In [ ]:
import pandas as pd
import torch
import time
import re # Импортируем модуль для регулярных выражений
from tqdm.auto import tqdm # Используем tqdm.auto для работы в разных средах

system_prompt = """Ты - ИИ для анализа финансового сентимента постов.
Твоя задача - классифицировать пост по его тональности относительно торговли акциями.
Верни ТОЛЬКО ОДНО ЧИСЛО: 1 (покупаем актив/бычий), 0 (нейтральный), -1 (продаем актив /медвежий).
Проанализируй следующий пост и верни ТОЛЬКО число."""

# --- Новая функция для YandexGPT с ручным промптом ---
def classify_sentiment_yandexgpt(
    df: pd.DataFrame,
    model,
    tokenizer,
    system_prompt: str,
    text_column: str = "processed_posts",
    output_column: str = "yandexgpt_sentiment",
    max_new_tokens: int = 5,
    default_sentiment: int = 0,
    prompt_format: str = "System:\n{system}\nUser:\n{user}\nAssistant:",
    # Альтернативный формат: "<|im_start|>system\n{system}<|im_end|>\n<|im_start|>user\n{user}<|im_end|>\n<|im_start|>assistant"
) -> pd.DataFrame:



    results = []

    device = model.device # Определяем устройство модели
    print(f"Начинаем обработку {len(df)} постов с моделью YandexGPT на устройстве {device}...")

    # Итерация по постам с использованием tqdm
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Классификация YandexGPT"):
        post_text = row[text_column]

        # Проверка на пустой или некорректный текст
        if not isinstance(post_text, str) or not post_text.strip():
            results.append(default_sentiment)
            continue

        full_prompt = f"""
                            System: {system_prompt}
                            User: {post_text}
                            Assistant:
                       """
        # # ---- Ключевое изменение: Ручное формирование промпта ----
        # full_prompt = prompt_format.format(system=system_prompt, user=post_text)
        # # ---------------------------------------------------------

        try:
            # Очистка кеша CUDA (может помочь при долгой работе)
            if device.type == 'cuda':
                 torch.cuda.empty_cache()

            messages = [
                # {"role": "system", "content": system_prompt},
                {"role": "user", "content": full_prompt}
            ]
            input_ids = tokenizer.apply_chat_template(
                messages, tokenize=True, return_tensors="pt"
            ).to("cuda")

            outputs = model.generate(*input_ids, max_new_tokens=5, do_sample=False, pad_token_id=tokenizer.pad_token_id)
            sent_score = tokenizer.decode(outputs[0][input_ids.size(1) :], skip_special_tokens=True)

            results.append(sent_score)

        except Exception as e:
            print(f"\nКритическая ошибка при обработке поста {index}: {e}")
            print(f"Текст поста: {post_text[:100]}...")
            print(f"Сформированный промпт: {full_prompt[:200]}...") # Помогает в отладке
            results.append(default_sentiment)

    df[output_column] = results
    return df


# # Создаем копию DataFrame для обработки
df_for_yandex_processing = posts_ygpt_9500.copy() # Замените my_dataframe на ваш DataFrame

print("Запуск классификации с YandexGPT...")
df_classified_yandex = classify_sentiment_yandexgpt(
    df=df_for_yandex_processing,
    model=model,         # Передаем модель YandexGPT
    tokenizer=tokenizer, # Передаем токенизатор YandexGPT
    system_prompt=system_prompt,
    text_column="processed_posts", # Укажите имя колонки с текстами
    output_column="yandex_8b_sentiment" # Укажите желаемое имя колонки для результатов
)

# --- Проверка результатов ---
print("\nПример результатов YandexGPT:")
print(df_classified_yandex.head())
print("\nРаспределение полученных классов YandexGPT:")
print(df_classified_yandex['yandex_8b_sentiment'].value_counts())
